In [1]:
import sys
sys.path.append("..")
from damage_indicator_module.damage_indicator_module import*

set_plot_formatting()

### INPUTS

In [2]:
# INPUT DATA OF THE BUILDING (manually uncomment desired building)

# building_id = 'ID1008'
# period = 0.95
# n_stories = 4
# n_floors_to_add = 1

# building_id = 'ID1012' # 'ID1012_bad'  'ID1012_good'
# period = 1.80
# n_stories = 8
# n_floors_to_add = 2

# building_id = 'ID1014'
# period = 2.15
# n_stories = 12
# n_floors_to_add = 3

building_id = 'ID1021'
period = 2.35
n_stories = 20
n_floors_to_add = 5

In [3]:
# INPUT MAINSHOCK-AFTERSHOCK ANALYSES CHARACTERISTICS
n_gm = 44
mainshock_scales = [0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
results_filename = '../0_Back_to_back_IDA_data/all_results_' + building_id + '.h5'

# INPUT DATA FOR DAMAGE INDICATOR CALCULATIONS
inspector_type = 'Probabilistic'
# n_floors_to_add = 1; # floors to sum damage index around peak story drift
                     # 4 story = 1, 8 story = 3, 12 story = 4, 20 story = 6

# Design parameters of the buildings site
Sms = 1.5*2/3
Sm1 = 0.9*2/3

### READING ANALYSIS DATA

#### Geometry and hinge capacity

In [4]:
with h5py.File(results_filename, 'r') as hf:
    # Get hinge capacity    
    building_group = '/building_metadata' 
    hinge_yield_rotation_positive = hf[building_group]['hinge_yield_rotation_positive'][:]
    hinge_cap_rotation_positive = hf[building_group]['hinge_cap_rotation_positive'][:]
    hinge_ultimate_rotation_positive = hf[building_group]['hinge_ultimate_rotation_positive'][:]
    hinge_yield_rotation_negative = hf[building_group]['hinge_yield_rotation_negative'][:]
    hinge_cap_rotation_negative = hf[building_group]['hinge_cap_rotation_negative'][:]
    hinge_ultimate_rotation_negative = hf[building_group]['hinge_ultimate_rotation_negative'][:]
    column_geometry = hf[building_group]['column_geometry'][:]
    
    key = '/intact_results/ida/collapse_intensities'
    collapse_intensities = pd.read_hdf(results_filename, key)

H_building = column_geometry[-1,0,1]

#### Collapse fragility data

In [5]:
n_scales = len(mainshock_scales) 

# Intact Collapse fragility
key = 'intact_results/ida/collapse_fragilities'
Savg_intact = pd.read_hdf(results_filename, key).loc['Sa_avg', 'Median']
beta_Savg_intact = pd.read_hdf(results_filename, key).loc['Sa_avg', 'Beta']
fragility_intact = pd.DataFrame({'Median': [Savg_intact], 'Beta': [beta_Savg_intact]})

# Get the median collapse capacity and other DI
j = 0
Savg_med_col = np.zeros([n_gm*n_scales])
beta_Savg_col = np.zeros([n_gm*n_scales])

for gm_i in range(n_gm):
    gm_id = 'GM'+str(gm_i+1) 

    for scale in mainshock_scales:
        key = '/mainshock_damage_results/' + gm_id + '/' + str(scale) + 'Col/ida/collapse_fragilities'
        colFragility = pd.read_hdf(results_filename, key)
        Savg_med_col[j] = colFragility.Median.Sa_avg
        beta_Savg_col[j] = colFragility.Beta.Sa_avg
        j = j + 1

# Create Dataframe with all collapse fragilities
fragilities = pd.DataFrame(np.column_stack((Savg_med_col, beta_Savg_col)), \
                           columns = ['Median', 'Beta'])

#### Mainshock characteristics

In [6]:
key = 'ground_motion_records/gm_record_metadata'
gm_metadata = pd.read_hdf(results_filename, key)

Sa_avgCol = gm_metadata['Intact Collapse Sa_avg']
Sa_Col = gm_metadata['Intact Collapse Sa(T1)']

Sa_avgMain = np.zeros(n_gm*n_scales)
Sa_Main = np.zeros(n_gm*n_scales)

gm_name = []
intensity_main = []

j = 0
for gm_i in range(n_gm):
    for scale in mainshock_scales:
        gm_name.append('GM' + str(gm_i))
        intensity_main.append(str(scale) + 'Col')
        Sa_avgMain[j] = Sa_avgCol[gm_i]*scale
        Sa_Main[j] = Sa_Col[gm_i]*scale
        j = j + 1

#### Building design intensity

In [7]:
sa_avg_design = design_sa_avg(period, Sms, Sm1)
sa_design = design_spectra([period], Sms, Sm1)[0]

sa_avg_design
sa_design

0.19083140217834815

0.2553191489361702

### COMPUTE DAMAGE INDICATORS

In [8]:
# Get various Damage Indicators for each damage instance

j = 0

# Initialize arrays
peak_story_drift = np.zeros([n_gm*n_scales,n_stories])
residual_story_drift = np.zeros([n_gm*n_scales,n_stories])
peak_drift_building = np.zeros([n_gm*n_scales]) 
residual_drift_building = np.zeros([n_gm*n_scales]) 

dsr_beams = np.zeros([n_gm*n_scales,4])
dsr_columns = np.zeros([n_gm*n_scales,4])

DI = np.zeros([n_gm*n_scales])
max_FDI = np.zeros(n_gm*n_scales)
FDI = np.zeros([n_gm*n_scales, n_stories+1])
DI_sdr = np.zeros([n_gm*n_scales])
DI_rsdr = np.zeros([n_gm*n_scales])
DI_bottom = np.zeros([n_gm*n_scales])
DI_bottom_sum = np.zeros([n_gm*n_scales])
FDI_max_bottom = np.zeros([n_gm*n_scales])
DI_columns = np.zeros([n_gm*n_scales])
DI_beams = np.zeros([n_gm*n_scales])
columnBool = np.zeros([n_gm*n_scales])

Sa_avgMainRatio = np.zeros([n_gm*n_scales])
Sa_MainRatio = np.zeros([n_gm*n_scales])

for gm_i in range(n_gm):
    gm_id = 'GM'+str(gm_i+1) 
    
    print(gm_id)
    
    for scale in mainshock_scales:
        print(scale)
        with h5py.File(results_filename, 'r') as hf:
    
            damaged_gm_scale_group = '/mainshock_damage_results/' + gm_id + '/' + str(scale) + 'Col/mainshock_edp'
                                   
            # Get damage indicators based on drift
            peak_story_drift[j, :] = hf[damaged_gm_scale_group]['peak_story_drift'][:]
            residual_story_drift[j, :] = hf[damaged_gm_scale_group]['residual_story_drift'][:]                        
            peak_drift_building[j] = max(abs(peak_story_drift[j]))
            residual_drift_building[j] = max(abs(residual_story_drift[j]))
                        
            # Get damage indicators based on fraction of damaged components
            building_group = '/mainshock_damage_results'
            peak_joint_pos = hf[damaged_gm_scale_group]['peak_joint_rotations_pos']
            peak_joint_neg = hf[damaged_gm_scale_group]['peak_joint_rotations_neg']            
           
            dsr_beams[j,:], dsr_columns[j,:] = get_dsr(peak_joint_pos, peak_joint_neg, hinge_yield_rotation_positive,\
                                             hinge_yield_rotation_negative, \
                                             hinge_cap_rotation_positive, \
                                             hinge_cap_rotation_negative, inspector_type)
            
            # Get damage indicators based on floor damage index      
            FDI[j,:], DI[j], DI_columns[j], DI_beams[j], columnBool[j] = compute_building_DI(peak_joint_pos,peak_joint_neg, hinge_yield_rotation_positive,\
                        hinge_cap_rotation_positive, hinge_yield_rotation_negative, hinge_cap_rotation_negative, inspector_type)
            max_FDI[j] = np.max(FDI[j,:])
            
            sdr_i = abs(peak_story_drift[j, :])
            fdi_i = FDI[j, :]
            indeces = sdr_i.argsort()
            DI_sdr[j] = sum_fdi_at_peak(indeces[-1], fdi_i, n_floors_to_add)
            
            rsdr_i = abs(residual_story_drift[j, :])
            indeces = rsdr_i.argsort()
            DI_rsdr[j] = sum_fdi_at_peak(indeces[-1], fdi_i, n_floors_to_add)
                        
            DI_bottom[j] = sum(fdi_i[1:n_floors_to_add+1])/n_floors_to_add
            
            DI_bottom_sum[j] = sum(fdi_i[1:n_floors_to_add+1])
            
            FDI_max_bottom[j] = max(fdi_i[1:n_floors_to_add+1])
            
            # Get intensity based damage indicators
            Sa_avgMainRatio[j] =  Sa_avgMain[j]/sa_avg_design
            Sa_MainRatio[j] =  Sa_Main[j]/sa_design
            
            j = j + 1

GM1
0.2
0.4
0.5
0.6


KeyboardInterrupt: 

### REDUCTION IN COLLAPSE CAPACITY

#### Reduction in median collapse capacity

In [8]:
k = Savg_med_col/Savg_intact

In [9]:
print(min(k))
print(np.argwhere(k < 0.1))

index_delete = np.argwhere(k < 0.1)
k[index_delete] = 0.1

# a = np.arange(start=0, stop=308, step=7)
# k2 = np.delete(k,a)
# k2.shape
# k[a]
# index_delete = np.argwhere(k2>1)
# k[index_delete].shape


0.15873920814294465
[]


### SAVE DAMAGE INDICATOR DATA

In [10]:
column_names = ['GM main', 'Scale main', 'Sa_avgMain', 'Sa_Main', 'Savg_med_col', 'beta_Savg_col', '$\kappa$', 
                '$SDR_{peak}$', '$RSDR_{peak}$', 'FDI', '$FDI_{peak}$', 'DI hinges', '$DI_{sdr}$', '$DI_{bottom}$', 
                '$DI_{bottom}^{sum}$','$FDI_{max}^{bottom}$','dsr_beams', 'dsr_columns', 'column_damage?', 
                'peak_story_drift', 'residual_story_drift', 
                '$Sa(T_1)_{avg}^{mainshock}/Sa(T_1)_{avg}^{DBE}$', '$Sa(T_1)^{mainshock}/Sa(T_1)^{DBE}$']

damage_instance_results = pd.DataFrame(list(zip(gm_name, intensity_main, Sa_avgMain, Sa_Main, Savg_med_col, 
                                                beta_Savg_col, k, peak_drift_building, residual_drift_building, 
                                                FDI, max_FDI, DI, DI_sdr, DI_bottom, DI_bottom_sum, FDI_max_bottom, 
                                                dsr_beams, dsr_columns, columnBool, 
                                                peak_story_drift, residual_story_drift, Sa_avgMainRatio, 
                                                Sa_MainRatio)), columns=column_names)

# UNCOMMENT THE APPROPIATE FILENAME
damage_instance_results.to_hdf('Damage_Instances_per_building.h5', key=building_id)
# damage_instance_results.to_hdf('Damage_Instances_' + building_id[0:6] + '_mat_props.h5', key=building_id)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['GM main', 'Scale main', 'FDI', 'dsr_beams', 'dsr_columns', 'peak_story_drift', 'residual_story_drift']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
